In [ ]:
import os, base64
from typing import List, Dict, Union
from openai import responses  # top-level helper (no explicit client needed)

def openai_response(
    prompt: str | List[Dict],                       # string or full message list
    model: str = "gpt-4o-mini",
    *,
    system_prompt: str | None = None,
    base64_image: str | None = None,                # embed one inline image if desired
    temperature: float = 1.0,
    reasoning_effort: str = "medium",               # for o1/o3/o4
    token_limit: int = 30_000,
) -> str:
    """
    Send a chat, vision, or o-family request to OpenAI and return the assistant's text.

    • `prompt` may be a plain string *or* an already-formed messages list.
    • If `base64_image` is supplied (and the model supports vision) it’s attached
      to the same user message.
    • The helper automatically picks the right token/effort field.

    Returns the assistant-message content as a string.
    """

    # --- hard caps for frontier models ------------------------------------------------
    if "gpt-4o" in model and token_limit > 16_384:
        token_limit = min(token_limit, 16_384)
    elif "gpt-4.1" in model and token_limit > 32_768:
        token_limit = min(token_limit, 32_768)

    # --- build message list -----------------------------------------------------------
    if isinstance(prompt, str):
        # convert to a minimal chat conversation
        user_content: List[Dict] = [{"type": "text", "text": prompt}]
        if base64_image and "o3-mini" not in model:   # o3-mini = text-only
            user_content.insert(
                0,
                {"type": "image_url",
                 "image_url": {"url": f"data:image/png;base64,{base64_image}"}}
            )
        messages: List[Dict] = [{"role": "user", "content": user_content}]
        if system_prompt:
            messages.insert(0, {"role": "system", "content": system_prompt})
    else:
        # user supplied full messages list
        messages = prompt

    # --- choose max-token field + optional reasoning ----------------------------------
    is_o_family = any(tag in model for tag in ("o1", "o3", "o4"))
    token_field  = "max_completion_tokens" if is_o_family else "max_tokens"

    params = {
        "model": model,
        "messages": messages,
        token_field: token_limit,
    }

    if is_o_family:
        params["reasoning"] = {"effort": reasoning_effort}
    else:
        params["temperature"] = temperature

    # --- call the endpoint ------------------------------------------------------------
    response = responses.create(api_key=os.getenv("OPENAI_API_KEY"), **params)
    return response.choices[0].message.content
